In [ ]:
### chan 2020/11/24

In [ ]:
### maximun pooling
def pooling_max_forward(x, poolKernel, strides=(2,2), padding=(0,0)):
    '''
    x: input feature map with shape(batchSize, channels, heights, weights)
    poolingKernel: pooling kernel
    
    z: output feature map after maximun pooling
    '''
    
    batchSize, channels, H, W = x.shape
    if padding != (0,0):
        x = np.lib.pad(x, ((0,0),(0,0),(padding[0],padding[0]),(padding[1],padding[1])), 'constant', constant_values=0)
    
    z_H = (H + 2*padding[0] - poolKernel[0]) // strides[0] + 1
    z_W = (W + 2*padding[0] - poolKernel[0]) // strides[0] + 1
    
    z = np.zeros((batchSize, channels, z_H, z_W))
    
    for n in np.arange(batchSize):
        for c in np.arange(channels):
            for h in np.arange(z_H):
                for w in np.arange(z_W):
                    z[n, c, h, w] = np.max(x[n, c, strides[0]*h : strides[0]*h+poolKernel[0], strides[1]*w : strides[1]*w+poolKernel[1]])
    
    return z

def pooling_max_backward(dzNext, x, poolKernel, strides=(2,2), padding=(0,0)):
    '''
    池化的反向传播，只是梯度的重新分配。比较容易理解，池化一般是不会有填零这一操作的。
    dzNext: error term of current layer, defined as dLoss(y,y*)/dz, where z is the output of current layer
    x: input feature map with shape(batchSize, channels, heights, weights)
    '''
    
    batchSize, channels, H, W = x.shape
    _, _, dzNext_H, dzNext_W = dzNext.shape
    
    ### padding zeros
    x = np.lib.pad(x, ((0, 0), (0, 0), (padding[0], padding[0]), (padding[1], padding[1])), 'constant', constant_values=0)
    dz = np.zeros_like(x)
    
    ### rearrange dzNext to dz, according to the position of maximum
    for n in arange(batchSize):
        for c in arange(channels):
            for h in arange(dzNext_H):
                for w in arange(dzNext_W):
                    flatten_idx = np.argmax(x[n, c,
                                                   strides[0]*h : strides[0]*h+poolKernel[0],
                                                   strides[1]*w : strides[1]*w+poolKernel[1]])
                    h_idx = strides[0]*i + flatten_idx // poolKernel[1]
                    w_idx = strides[1]*j + flatten_idx % poolKernel[1]
                    dz[n, c, h_idx, w_idx] += dzNext[n, c, dzNext_H, dzNext_W]
    ### remove padding
    if padding[0] > 0 and padding[1] > 0:
        dz = dz[:, :, padding[0]:-padding[0], padding[1]:-padding[1]]
    elif padding[0]>0:
        dz = dz[:, :, padding[0]:-padding[0], :]
    elif padding[1]>0:
        dz = dz[:, :, :, padding[1]:-padding[1]]
    return dz

In [ ]:
### average pooling
def pooling_average_forward(x, poolKernel, strides=(2,2), padding=(0,0)):
    '''
    x: input feature map with shape(batchSize, channels, heights, weights)
    poolingKernel: pooling kernel
    
    z: output feature map after average pooling
    '''
    
    batchSize, channels, H, W = x.shape
    if padding != (0,0):
        x = np.lib.pad(x, ((0,0),(0,0),(padding[0],padding[0]),(padding[1],padding[1])), 'constant', constant_values=0)
    
    z_H = (H + 2*padding[0] - poolKernel[0]) // strides[0] + 1
    z_W = (W + 2*padding[0] - poolKernel[0]) // strides[0] + 1
    
    z = np.zeros((batchSize, channels, z_H, z_W))
    
    for n in np.arange(batchSize):
        for c in np.arange(channels):
            for h in np.arange(z_H):
                for w in np.arange(z_W):
                    z[n, c, h, w] = np.mean(x[n, c, strides[0]*h:strides[0]*h+poolKernel[0], strides[1]*w:strides[1]*w+poolKernel[1]])
    
    return z

def pooling_average_backward(dzNext, x, poolKernel, strides=(2,2), padding=(0,0)):
    '''
    the backpropagation of pooling is the rearranement of dzNext, it has no paraments to update.
    And, there is no padding in pooling usually.
    dzNext: error term of current layer, defined as dLoss(y,y*)/dz, where z is the output of current layer
    x: input feature map with shape(batchSize, channels, heights, weights)
    '''
    
    batchSize, channels, H, W = x.shape
    _, _, dzNext_H, dzNext_W = dzNext.shape
    
    ### padding zeros
    x = np.lib.pad(x, ((0, 0), (0, 0), (padding[0], padding[0]), (padding[1], padding[1])), 'constant', constant_values=0)
    dz = np.zeros_like(x)
    
    ### rearrange dzNext to dz
    for n in arange(batchSize):
        for c in arange(channels):
            for h in arange(dzNext_H):
                for w in arange(dzNext_W):
                    dz[n, c, strides[0]*h : strides[0]*h+poolKernel[0], strides[1]*w : strides[1]*w+poolKernel[1]] +=
                    dzNext[n, c, dzNext_H, dzNext_W] / (poolKernel[0]*poolKernel[1])
                    
    ### remove padding
    if padding[0] > 0 and padding[1] > 0:
        dz = dz[:, :, padding[0]:-padding[0], padding[1]:-padding[1]]
    elif padding[0]>0:
        dz = dz[:, :, padding[0]:-padding[0], :]
    elif padding[1]>0:
        dz = dz[:, :, :, padding[1]:-padding[1]]
        
    return dz

In [ ]:
### global average pooling
def pooling_globalAverage_forward(x):
    '''
    global pooling output a value each feature map
    x: input feature map with shape(batchSize, channels, heights, weights)
    z: output feature map wih shape(batchSize, channels)
    '''
    z = np.mean(np.mean(x, axis=-1), axis=-1)
    return z

def pooling_globalAverage_backward(dzNext, dz, x):
    '''
    dzNext: error term of current layer, defined as dLoss(y,y*)/dz, where z is the output of current layer
    x: input feature map with shape(batchSize, channels, heights, weights)
    
    dz: error term of layer before dzNext
    '''
    batchSize, channels, H, W = x.shape
    
    dz = np.zeros_like(x)
    
    ### rearrange dzNext to dz
    for n in arange(batchSize):
        for c in arange(channels):
            dz[n, c, :, :] = dzNext[n,c] / H*W
    return dz

In [1]:
import numpy as np
def flatten_forward(z):
    """
    将多维数组打平，前向传播
    :param z: 多维数组,形状(N,d1,d2,..)
    :return:
    """
    N = z.shape[0]
    return np.reshape(z, (N, -1))


def flatten_backward(next_dz, z):
    """
    打平层反向传播
    :param next_dz:
    :param z:
    :return:
    """
    return np.reshape(next_dz, z.shape)

x = np.random.randn(2, 3, 20, 20).astype(np.float64)
print(flatten_forward(x).shape)

(2, 1200)
